In [1]:
import numpy as np
import itertools
import sys
sys.path.insert(0, '../CClassCSemigroups/')

In [2]:
import CsemigroupsCPP as CPP

In [3]:
def ComputeCone(bound,eq):
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq):
            cone.append(list(x))
    return cone

In [4]:
def PrecGradLex(a,b): # Check if A<B\n",
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x > 0:
                return True
            if x < 0:
             return False

In [5]:
def PrecGradRevLex(a,b): # Check if A<B\n",
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        c = list(c)
        c.reverse()
        for x in c:
            if x < 0:
                return True
            if x > 0:
             return False

In [6]:
def MaximumOrder(v,order):
    maximum = v[0]
    for x in v:
        if order(maximum,x):
            maximum = x
    return maximum

In [7]:
def GetSemigroupFromGaps(gaps,eq,order): # Note: Last gap is the frobenius element.
    frob = MaximumOrder(gaps,order)
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq) and not CPP.allZero(list(x)):
            cone.append(list(x))
    for x in gaps:
        cone.remove(x)
    cs = CPP.Csemigroup(cone)
    return cs

In [8]:
def ComputeIC(cone,frob,eq):
    aux = []
    for x in cone:
        if CPP.belongCone([frob[i]-x[i] for i in range(len(x))],eq):
            aux.append(x)
    return aux

In [9]:
def ComputeRootTreeIrreductible(IC, frob, order,eq):
    hFrob = [x/2 for x in frob]
    i1 = []
    for x in IC:
        if not CPP.allZero(list(x)) and not order(hFrob,x):
            i1.append(x)
    i1.append(frob)
    return GetSemigroupFromGaps(i1,eq,order)

In [10]:
def GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order):
    hFrob = [x/2 for x in frob]
    i2 = []
    for x in gen:
        aux = list(np.array(frob)-np.array(x))
        if order(hFrob,x) and CPP.allPositive([frob[i]-x[i] for i in range(len(x))]):
            i2.append(x)
    return i2

In [11]:
def CheckPropertiesGeneratorsTree(gen, frob,IC,order):
    aux = []
    candidates = GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order)
    for x in candidates:
        if x in IC and not CPP.belongByGens([2*x[i]-frob[i] for i in range(len(x))],gen) and [3*x[i] for i in range(len(x))] != [2*frob[i] for i in range(len(x))] and [4*x[i] for i in range(len(x))] != [3*frob[i] for i in range(len(x))]:
            aux.append(x)
    return aux

In [12]:
def ComputeIrreducibleChildren(sg,frob,order, ic,eq):
    children = []
    A = CheckPropertiesGeneratorsTree(sg.GetGenerators(),frob,ic,order)
    print("A =",A)
    for x in A:
        gaps = list(sg.GetGaps())
        gaps.append(x)
        gaps.remove([frob[i]-x[i] for i in range(len(x))])
        children.append(GetSemigroupFromGaps(gaps,eq,order))
    return children

In [13]:
def ComputeTreeIrreducible(frob, eq, order):
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = ComputeCone(bound,eq)
    ic = ComputeIC(cone,frob,eq)
    sf = ComputeRootTreeIrreductible(ic, frob, order,eq)
    X = [sf]
    Y = []
    while X != []:
        print(len(X))
        for sg in X:
            print("Huecos padre:",sg.GetGaps())
            print("Huecos hijos:")
            children = ComputeIrreducibleChildren(sg,frob,order, ic,eq)
            for kk in children:
                print(kk.GetGaps())
            Y.append([sg,children])
        X = children
        
        print("En X hay:")
        for kk in X:
            print(kk.GetGenerators())
        print("\n")
    return(Y)

In [14]:
cone = CPP.Csemigroup([[1,0],[1,1]])
cone.GetHyperplanes()

[[0, 1], [1, -1]]

In [15]:
cone = CPP.Csemigroup([[1,0,0],[0,0,1],[1,1,1],[1,2,2]])
cone.GetHyperplanes()
#cone.GetGenerators()

[[0, -1, 1], [0, 1, 0], [2, -1, 0]]

### Ejemplo 3D

In [29]:
eq = [[0, -1, 1], [0, 1, 0], [1, -1, 0]]
frob = [4,2,3]
cone = ComputeCone([3*x for x in frob],eq)
ic = ComputeIC(cone,frob,eq)
sf1 = ComputeRootTreeIrreductible(ic, frob, PrecGradLex,eq)
aa = sf1.GetGenerators()

In [30]:
eq = [[0, -1, 1], [0, 1, 0], [1, -1, 0]]
frob = [4,2,3]
cone = ComputeCone([3*x for x in frob],eq)
ic = ComputeIC(cone,frob,eq)
sf1 = ComputeRootTreeIrreductible(ic, frob, PrecGradRevLex,eq)
bb = sf1.GetGenerators()

In [31]:
aa == bb

True

In [ ]:
eq = [[0, -1, 1], [0, 1, 0], [1, -1, 0]]
frob = [12,6,12]
cone = ComputeCone([3*x for x in frob],eq)
ic = ComputeIC(cone,frob,eq)
sf1 = ComputeRootTreeIrreductible(ic, frob, PrecGradLex,eq)
aa = sf1.GetGenerators()
# sf1.GetGaps()

In [ ]:
eq = [[0, -1, 1], [0, 1, 0], [1, -1, 0]]
frob = [12,6,12]
cone = ComputeCone([3*x for x in frob],eq)
ic = ComputeIC(cone,frob,eq)
sf2 = ComputeRootTreeIrreductible(ic, frob, PrecGradRevLex,eq)
bb = sf2.GetGenerators()
# sf2.GetGaps()

In [ ]:
aa==bb

In [ ]:
aa

In [ ]:
ic

In [ ]:
PrecGradLex([2,1,1],[1,1,2])

In [ ]:
PrecGradLex([2,2,4],[1,4,3])

In [ ]:
PrecGradRevLex([2,2,4],[1,4,3])

In [ ]:
arbol3d1 = ComputeTreeIrreducible([3,2,3], [[0, -1, 1], [0, 1, 0], [1, -1, 0]], PrecGradLex)

In [ ]:
arbol3d2 = ComputeTreeIrreducible([3,2,3], [[0, -1, 1], [0, 1, 0], [1, -1, 0]], PrecGradRevLex)

In [ ]:
for x in arbol3d1:
    print("Padre",x[0].GetGenerators())
    print("Hijos:")
    for y in x[1]:
        print("   ->",y.GetGenerators(),len(y.GetGaps()))
    print("\n")

### Ejemplos 2D 

In [41]:
eq = [[0, 1], [1, -1]]
frob = [4,2]
cone = ComputeCone([24,18],eq)
ic = ComputeIC(cone,frob,eq)
sf = ComputeRootTreeIrreductible(cone, frob, PrecGradRevLex,eq)
ic = ComputeIC(cone,frob,eq)
aa = sf.GetGenerators()

In [42]:
ic

[[0, 0], [1, 0], [1, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [4, 2]]

In [43]:
eq = [[0, 1], [1, -1]]
frob = [4,2]
cone = ComputeCone([24,18],eq)
sf = ComputeRootTreeIrreductible(cone, frob, PrecGradLex,eq)
ic = ComputeIC(cone,frob,eq)
bb = sf.GetGenerators()

In [44]:
aa

[[3, 0],
 [4, 0],
 [5, 0],
 [3, 1],
 [4, 1],
 [5, 1],
 [2, 2],
 [3, 2],
 [3, 3],
 [4, 3]]

In [ ]:
sf.GetHyperplanes()

In [34]:
aa == bb

True

In [ ]:
sf.GetGaps()

In [ ]:
sf.GetPseudoFrobenius()

In [ ]:
children = ComputeIrreducibleChildren(sf,frob,PrecGradRevLex, ic,eq)

In [ ]:
children[0].GetPseudoFrobenius()

In [45]:
arbol  = ComputeTreeIrreducible([4,2], [[0, 1], [1, -1]], PrecGradLex)

1
Huecos padre: [[1, 0], [1, 1], [2, 0], [2, 1], [4, 2]]
Huecos hijos:
A = [[3, 1], [2, 2]]
[[1, 0], [2, 0], [2, 1], [3, 1], [4, 2]]
[[1, 0], [1, 1], [2, 1], [2, 2], [4, 2]]
En X hay:
[[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]]
[[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]


2
Huecos padre: [[1, 0], [2, 0], [2, 1], [3, 1], [4, 2]]
Huecos hijos:
A = []
Huecos padre: [[1, 0], [1, 1], [2, 1], [2, 2], [4, 2]]
Huecos hijos:
A = [[3, 2]]
[[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
En X hay:
[[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]]


1
Huecos padre: [[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
Huecos hijos:
A = []
En X hay:




In [46]:
for x in arbol:
    print("Padre",x[0].GetGenerators())
    print("Hijos:")
    for y in x[1]:
        print("   ->",y.GetGenerators(),len(y.GetGaps()))

Padre [[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [3, 3], [4, 3]]
Hijos:
   -> [[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]] 5
   -> [[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]] 5
Padre [[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]]
Hijos:
Padre [[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]
Hijos:
   -> [[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]] 5
Padre [[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]]
Hijos:


In [ ]:
arbol2  = ComputeTreeIrreducible([4,2], [[0, 1], [1, -1]], PrecGradRevLex)

In [ ]:
for x in arbol2:
    print("Padre",x[0].GetGenerators())
    print("Hijos:")
    for y in x[1]:
        print("   ->",y.GetGenerators(),len(y.GetGaps()))